In [2]:
import sys
sys.path.append('..')

import pandas as pd
from src.audio_utils import get_audio_features
from tqdm import tqdm

SyntaxError: invalid syntax (audio_utils.py, line 32)

In [ ]:
csv_path = "../data/chartex_final.csv"

sorting data by popularity

In [ ]:
# # Load the CSV file into a DataFrame
# df = pd.read_csv(csv_path)

# # Drop the 'number_of_videos_last_14days' and 'song_name' columns
# df = df.drop(columns=['Unnamed: 0'])

# # Sort the DataFrame by the 'track_name' column
# sorted_df = df.sort_values(by='total_likes_count', ascending=False)

# # Reset the index of the sorted DataFrame
# sorted_df = sorted_df.reset_index(drop=True)

# # Overwrite the original CSV file with the sorted data
# sorted_df.to_csv(csv_path, index=False)

# # Print a message to confirm that the file has output_directory overwritten
# print(f'The file {csv_path} has been overwritten with the sorted data.')

getting Spotify data for most popular song

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Get the first row of data
example_track_data = df.iloc[1]

# Print the resulting row
print(example_track_data)

track_name           Spongebob Tomfoolery - Dante9k Remix
track_pop                                              33
artist                                        David Snell
artist_pop                                             12
album                Spongebob Tomfoolery (Dante9k Remix)
danceability                                        0.831
energy                                              0.445
key                                                   0.0
loudness                                          -14.492
mode                                                  1.0
speechiness                                         0.193
acousticness                                       0.0257
instrumentalness                                    0.908
liveness                                            0.113
valence                                             0.508
tempo                                             125.015
id                                 5RUBtOzrUcM0tIRozgRtzA
duration_ms   

extracting audio features

In [ ]:
%%time

audio_dir = "../data/track_downloads"

# Get the additional audio features for the track
features = get_audio_features(example_track_data, audio_dir)

Found youtube url: https://www.youtube.com/watch?v=4qXVIbs4CR0&pp=ygU3RGF2aWQgU25lbGwgU3BvbmdlYm9iIFRvbWZvb2xlcnkgLSBEYW50ZTlrIFJlbWl4IGx5cmljcw%3D%3D
Downloaded audio to ../data/track_downloads/David Snell - Spongebob Tomfoolery - Dante9k Remix.wav


/Users/omersideman/dev/technion/ai-project/notebooks/../src/audio_utils.py:34: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, mono=True, duration=30)
/Users/omersideman/dev/technion/ai-project/venv/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Extracted features: {'chroma_stft': 0.29328093, 'rmse': 0.057167627, 'spec_cent': 1877.152916007926, 'spec_bw': 1771.1224763016426, 'rolloff': 3266.2469161184213, 'zcr': 0.10765354404508513, 'mfcc': -12.507167}
CPU times: user 2.86 s, sys: 665 ms, total: 3.53 s
Wall time: 6.22 s


testing on a batch of songs

In [ ]:
def batch_download(df, start_index, end_index, audio_dir):
    for i in range(start_index, end_index):
        print(f'Processing track {i+1+start_index} of {len(df)}')
        track_data = df.iloc[i]
        features = get_audio_features(track_data)
        # add features to dataframe
        for feature_names in features.keys():
            df.loc[i, feature_names] = features[feature_names]
        # save dataframe  
        df.to_csv(csv_path, index=False)
    

for i in range(190, len(df), 10):
    batch_download(df, i, i+10, audio_dir)
    

Processing track 381 of 3932
Found youtube url: https://www.youtube.com/watch?v=jfKfPfyJRdk&pp=ygU0TG9GaSBCeSBOYXR1cmUgTG9maSBIaXAgSG9wICYgTmF0dXJlIEFtYmlhbmNlIGx5cmljcw%3D%3D


LiveStreamError: jfKfPfyJRdk is streaming live and cannot be loaded

In [ ]:
df.head()

,track_name,track_pop,artist,artist_pop,album,danceability,energy,key,loudness,mode,...,artist_name,total_likes_count,number_of_videos,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc
0,Love You So,0,The King Khan & BBQ Show,37,The King Khan & BBQ Show,0.389,0.896,5.0,-2.622,1.0,...,The King Khan & BBQ Show,4187998484,33542864,0.390025,0.326011,1795.164660,2164.226937,3672.819458,0.069522,2.650419
1,Spongebob Tomfoolery - Dante9k Remix,33,David Snell,12,Spongebob Tomfoolery (Dante9k Remix),0.831,0.445,0.0,-14.492,1.0,...,David Snell,1590913470,6200000,0.293281,0.057168,1877.152916,1771.122476,3266.246916,0.107654,-12.507167
2,SugarCrash!,74,ElyOtto,53,SugarCrash!,0.748,0.734,0.0,-6.499,1.0,...,ElyOtto,1548567686,4700000,0.293770,0.132078,1944.135169,1843.284434,3800.935350,0.097148,-11.325932
3,Home,75,Edith Whiskers,59,Home,0.725,0.199,2.0,-12.478,1.0,...,Tom Rosenthal,1536455117,1700000,0.275464,0.099697,864.490292,998.240419,1515.407503,0.044328,-7.585949
4,Blue Blood,7,Orchestra Heinz Kiessling,13,High Society,0.678,0.508,0.0,-13.403,1.0,...,Heinz Kiessling,1392234379,1100000,0.355247,0.092926,1683.105946,2114.052010,3641.252982,0.061494,-1.111346
